<a href="https://colab.research.google.com/github/nehaal10/license-plate-detection/blob/main/wheat_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **importing required files**

In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import ast
!pip install wandb
import wandb

     |████████████████████████████████| 1.7 MB 4.8 MB/s 
     |████████████████████████████████| 180 kB 52.2 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 143 kB 59.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=985ff8acf47ba22a32e3e314babe76c1a8985ebd312f63a883bd677172f4750f
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3182748365b72c42e8db57dd3982ea36f057dbb2e2106777e71f34dcce3cc92d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


# **CSV TO YOLO TEXT**

In [ ]:
from numpy.lib.npyio import savetxt
def yolo_format(df,data_type):
  for _,row in tqdm(df.iterrows(),total=len(df)):
    image_name=row['image_id']
    bbox=row['bbox']
    yolo_data=[]
    for i in bbox:
      x=i[0]
      y=i[1]
      w=i[2]
      h=i[3]

      x= x + (w/2)
      y= y+ (h/2)

      x/=1024.0
      y/=1024.0
      w/=1024.0
      h/=1024.0

      yolo_data.append([0,x,y,w,h])
    yolo_data=np.array(yolo_data)
    np.savetxt(
        '/content/drive/MyDrive/yolo_models/wheat_recognision/{}/labels/{}.txt'.format(data_type,image_name),
        yolo_data,
        fmt=["%d","%f","%f","%f","%f"]
    )
    
    shutil.copy('/content/drive/MyDrive/kaggle/{}.jpg'.format(image_name),'/content/drive/MyDrive/yolo_models/wheat_recognision/{}/images'.format(data_type))
    



df=pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
df['bbox']=df['bbox'].apply(ast.literal_eval)
df=df.groupby('image_id')['bbox'].apply(list).reset_index(name='bbox')
train_df,test_df=train_test_split(df,test_size=0.2,shuffle=True,random_state=42)

In [ ]:
yolo_format(train_df,'train')

100%|██████████| 2698/2698 [01:15<00:00, 35.67it/s]


In [ ]:
yolo_format(test_df,'valid')

100%|██████████| 675/675 [00:16<00:00, 40.22it/s]


# **clonning github repo**

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10752, done.
remote: Total 10752 (delta 0), reused 0 (delta 0), pack-reused 10752
Receiving objects: 100% (10752/10752), 10.83 MiB | 7.74 MiB/s, done.
Resolving deltas: 100% (7434/7434), done.


# **creating custom yaml file **

In [ ]:
li=['wheat']
with open('wheat.yaml','w') as f:
  f.write("train : {}\n".format('/content/drive/MyDrive/yolo_models/wheat_recognision/train/images'))
  f.write("val : {}\n".format('/content/drive/MyDrive/yolo_models/wheat_recognision/valid/images'))
  f.write("nc : {}\n\n".format(len(li)))
  for i in li:
    f.write("names : ['{}']".format(i))

# **downloading pretrained weights**

In [ ]:
os.chdir('/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5')

In [ ]:
os.popen('sh data/scripts/download_weights.sh')

# **Installing required dependseries**

In [ ]:
!pip install -r requirements.txt

In [ ]:
!nvidia-smi

Mon Jan 31 04:47:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **loging in to wandb account to veiw real training**

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!python train.py --img 1024 --batch 8 --epochs 500 --data /content/drive/MyDrive/yolo_models/wheat_recognision/wheat.yaml --cfg models/hub/yolov5s-transformer.yaml --weights yolov5s_model_wandb/yolov5s/weights/last.pt  --hyp data/hyps/hyp.scratch-low.yaml --project yolov5s_model_wandb --name yolov5s --cache

Traceback (most recent call last):
  File "train.py", line 41, in <module>
    import val  # for end-of-epoch mAP
  File "/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5/val.py", line 38, in <module>
    from models.common import DetectMultiBackend
  File "/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5/models/common.py", line 24, in <module>
    from utils.datasets import exif_transpose, letterbox
  File "/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5/utils/datasets.py", line 29, in <module>
    from utils.augmentations import Albumentations, augment_hsv, copy_paste, letterbox, mixup, random_perspective
  File "/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5/utils/augmentations.py", line 12, in <module>
    from utils.general import LOGGER, check_version, colorstr, resample_segments, segment2box
  File "/content/drive/MyDrive/yolo_models/wheat_recognision/yolov5/utils/general.py", line 33, in <module>
    from utils.metrics import box_i

In [ ]:
!python detect.py --weights yolov5s_model_wandb/yolov5s2/weights/best.pt --source /content/drive/MyDrive/yolo_models/wheat_recognision/t/images --conf-thres 0.5 --iou-thres 0.65

detect: weights=['yolov5s_model_wandb/yolov5s2/weights/best.pt'], source=/content/drive/MyDrive/yolo_models/wheat_recognision/t/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.65, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-218-g7539cd7 torch 1.8.1+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 216 layers, 7013590 parameters, 0 gradients, 15.6 GFLOPs
image 1/1001 /content/drive/MyDrive/yolo_models/wheat_recognision/t/images/005b0d8bb.jpg: 640x640 22 wheats, Done. (0.017s)
image 2/1001 /content/drive/MyDrive/yolo_models/wheat_recognision/t/images/00764ad5d.jpg: 640x640 35 wheats, Done. (0.015s)
image 3/1001 /content/drive/MyDrive/yolo_models/wheat_recognision/t/imag